# Intraday vs Overnight Momentum

- Momentum is a widely used trading strategy in quantitative finance. The core idea is that assets that have performed well in the past will continue to perform well in the near future. We are now going to think a little bit about past literature and create a better version of momentum. 
- There is an observation that overnight and intraday returns diﬀer. The paper "Tug of War: Overnight versus Intraday Expected Returns" by Polk, Sapienza, and Subrahmanyam explores how expected returns manifest diﬀerently during overnight and intraday periods.

## Information Events


### Earnings Announcements
- Earnings are typically announced outside of market hours, either **after market close** or **before market open**.

### Macro Announcements
- Major macroeconomic data like **Non-Farm Payroll** report are typically announced **before the market opens**, usually at **8:30 AM EST** on the first Friday of every month.
- Other macro data releases such as **CPI (Consumer Price Index)**, **GDP**, and **interest rate decisions** can be announced before or during market hours depending on the timing.
- Macro announcements can have a broad impact on investor sentiment and expectations for the overall economy. Certain sectors can be directly impacted by macroeconomic news.

## Loading Data

In [33]:
import pandas as pd
import numpy as np
import os
import warnings
warnings.filterwarnings('ignore')

if os.path.exists('../data') == True:
    data_folder = os.path.abspath('../data')
    os.chdir(data_folder)
    print(f'Successfully connected to data')
else:
    print('invalid data path!')

Successfully connected to data


In [2]:
mom_df = pd.read_parquet('hw2_mfin7037_data.parquet')
mom_df['date'] = pd.to_datetime(mom_df['date'])
mom_df

,permno,date,ret,intraday_ret_month,overnight_ret_month,mcap_lag1,prc_lag1,mom_intraday,mom,mom_overnight,mcap_bin
0,10061.0,1986-02-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10061.0,1986-03-31,-0.034014,NaN,NaN,20635.125000,18.3750,NaN,NaN,NaN,4.0
2,10061.0,1986-04-30,-0.140845,NaN,NaN,19933.250000,17.7500,NaN,NaN,NaN,4.0
3,10061.0,1986-05-30,-0.016393,NaN,NaN,17125.750000,15.2500,NaN,-0.034606,NaN,4.0
4,10061.0,1986-06-30,-0.033333,NaN,NaN,16845.000000,15.0000,NaN,-0.186412,NaN,3.0
...,...,...,...,...,...,...,...,...,...,...,...
3776102,93399.0,2018-09-28,-0.095432,-0.162727,0.125750,21536.024646,0.8844,-0.305023,-0.207639,0.097378,1.0
3776103,93399.0,2018-10-31,-0.223750,0.238409,-0.373188,19480.800290,0.8000,-0.755411,-0.389662,0.396091,1.0
3776104,93399.0,2018-11-30,-0.444605,-0.560893,0.264830,15125.075803,0.6210,-0.802384,-0.255804,0.618065,1.0
3776105,93399.0,2018-12-31,-0.478110,0.074360,-0.514231,8400.384693,0.3449,-0.838701,-0.603673,0.306514,1.0


## Analysis

### Intraday/Overnight Momentum Correlation

In [3]:
from tabulate import tabulate

# Calculate correlation matrix
corr_matrix = mom_df[['mom_intraday', 'mom', 'mom_overnight']].corr()
print("Correlation Matrix:")
print(tabulate(corr_matrix, headers='keys', tablefmt='rounded_grid'))

Correlation Matrix:
╭───────────────┬────────────────┬──────────┬─────────────────╮
│               │   mom_intraday │      mom │   mom_overnight │
├───────────────┼────────────────┼──────────┼─────────────────┤
│ mom_intraday  │       1        │ 0.576277 │       -0.655101 │
├───────────────┼────────────────┼──────────┼─────────────────┤
│ mom           │       0.576277 │ 1        │        0.134288 │
├───────────────┼────────────────┼──────────┼─────────────────┤
│ mom_overnight │      -0.655101 │ 0.134288 │        1        │
╰───────────────┴────────────────┴──────────┴─────────────────╯


### Data Filtering
- Kick out anything in the bottom 20% of market capitalisation.
- Kick out anything with a price less than 5 at that time.
- At least 8 past returns (implied in the paper Daniel and Moskowitz (2013))

In [4]:
# number of missing values by grouping
mom_df['rollvalidobs'] = (
    mom_df
    .assign(boolean_retnotnull=mom_df['ret'].notnull())
    .groupby('permno')['boolean_retnotnull']
    .rolling(11) # rolling sum
    .sum()
).reset_index([0], drop=True)

# filtering
filtered_mom_df = mom_df[(
    (mom_df['mcap_bin'] > 2) &
    (mom_df['prc_lag1'] >= 5) &
    (mom_df['rollvalidobs'] >= 8)
)]

### Report Tables
- Based on three factors: MOM, MOM_Intraday, MOM_Overnight, run analysis
- Generate report tables: for each factor, run Equal-weighted and Value-weighted portfolios on:
    - Daily return
    - Intraday return
    - Overnight return
- Also construct long/short portfolios
- Report mean portfolio returns and T-statistics.

In [ ]:
# --- Helper: Apply quantiles within each date group ---
def apply_quantiles(df, col, bins=10):
    """
    Assigns a quantile-based bin (1,...,bins) for each value in `col`
    within each date group.
    """
    def quantile_bin(s):
        # Use pd.qcut to get quantile bins; if duplicate edges occur, use rank instead.
        try:
            return pd.qcut(s, q=bins, labels=False, duplicates="drop") + 1
        except Exception:
            return np.ceil(s.rank(method='average') / len(s) * bins)
    return df.groupby('date')[col].transform(quantile_bin)

In [ ]:
# apply quantiles for each factor: (vanilla) momentum, intraday momentum, overnight momentum
for factor in ['mom', 'mom_intraday', 'mom_overnight']:
    filtered_mom_df[f'{factor}_bin'] = apply_quantiles(filtered_mom_df, factor)

In [ ]:
def analyze_strategy(data, factor):
    """
    Analyze the performance of a factor by creating tables.
    The function computes the average returns (including daily return, intraday return and overnight return)
    and t-statistics for each quantile of the factor, using both equal-weighted and value-weighted methods.
    """

    # step 1: form portfolios using both EW and VW
    portfolios_ret = (
        data
        .groupby(['date', f'{factor}_bin'])
        .apply(
            lambda g: pd.Series({
                'EW': g['ret'].mean(),
                'VW': (g['ret'] * g['mcap_lag1']).sum() / g['mcap_lag1'].sum(),
                'EW_intraday': g['intraday_ret_month'].mean(),
                'VW_intraday': (g['intraday_ret_month'] * g['mcap_lag1']).sum() / g['mcap_lag1'].sum(),
                'EW_overnight': g['overnight_ret_month'].mean(),
                'VW_overnight': (g['overnight_ret_month'] * g['mcap_lag1']).sum() / g['mcap_lag1'].sum()
            }))
        .T.stack(0)
    )
    portfolios_ret['10-1'] = portfolios_ret[10] - portfolios_ret[1]

    # step 2: calculate average returns of all portfolios
    ret_mean = portfolios_ret.groupby(level=0).mean()

    # generating t-statistics table
    ret_tstat = (
        portfolios_ret
        .groupby(level=0)
        .apply(lambda x: x.mean()/x.std()*np.sqrt(len(x)))
    )
    
    return {
        'ret_mean': ret_mean,
        'ret_tstat': ret_tstat
    }

In [ ]:
# run analysis on all factors
for factor in ['mom', 'mom_intraday', 'mom_overnight']:
    result = analyze_strategy(filtered_mom_df, factor)
    
    # format as percentages
    formatted_ret_mean = result['ret_mean'].applymap(lambda x: f"{x:.2%}")
    print(f"\n{factor.upper()} - Mean Returns")
    print(tabulate(formatted_ret_mean, headers='keys', tablefmt='rounded_grid'))
    
    # format as percentages
    formatted_ret_tstat = result['ret_tstat'].applymap(lambda x: f"{x:.2f}")
    print(f"\n{factor.upper()} - T-Statistics")
    print(tabulate(formatted_ret_tstat, headers='keys', tablefmt='rounded_grid'))


MOM - Mean Returns
╭──────────────┬────────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬────────┬────────┬────────╮
│              │ 1.0    │ 2.0   │ 3.0   │ 4.0   │ 5.0   │ 6.0   │ 7.0   │ 8.0   │ 9.0    │ 10.0   │ 10-1   │
├──────────────┼────────┼───────┼───────┼───────┼───────┼───────┼───────┼───────┼────────┼────────┼────────┤
│ EW           │ 0.37%  │ 0.91% │ 1.00% │ 1.02% │ 1.10% │ 1.15% │ 1.13% │ 1.17% │ 1.26%  │ 1.50%  │ 1.12%  │
├──────────────┼────────┼───────┼───────┼───────┼───────┼───────┼───────┼───────┼────────┼────────┼────────┤
│ EW_intraday  │ -0.49% │ 0.33% │ 0.47% │ 0.45% │ 0.49% │ 0.56% │ 0.43% │ 0.37% │ 0.30%  │ 0.10%  │ 0.58%  │
├──────────────┼────────┼───────┼───────┼───────┼───────┼───────┼───────┼───────┼────────┼────────┼────────┤
│ EW_overnight │ 1.15%  │ 0.60% │ 0.48% │ 0.52% │ 0.49% │ 0.54% │ 0.67% │ 0.76% │ 0.94%  │ 1.52%  │ 0.38%  │
├──────────────┼────────┼───────┼───────┼───────┼───────┼───────┼───────┼───────┼────────┼────────┼────────┤